In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
import sys
import glob
import re
import requests
from matplotlib.patches import Rectangle
from datetime import datetime
# sns.set()

# Introduction <a id='intro'></a>

This notebook cleans and wrangles numerous data sets, making them uniform
so that they can be used in a data-driven model for COVID-19 prediction.

The key cleaning measures are those which find the most viable set of countries and date ranges
such that the maximal amount of data can be used. In other words, different datasets can have data
on a different set of countries; to avoid introducing large quantities of missing values
the intersection of these countries is taken. For the date ranges, depending on the quantity,
extrapolation/interpolation is used to ensure that each time series is defined to be non-zero
on all dates. This process is kept track of by encoding the dates which have interpolated values.
There are two measures to do so. Essentially its one hot encoding for the categories ['extrapolated', 'interpolated', 'actual']. The other measure is to track the "days since infection" where 0 represents the first day with a recorded
case of COVID within that country. I leave the more complex feature creation to the exploratory data analysis portion
of this project.

Some of the data is currently not used but may be incorporated later on.


# Table of contents<a id='toc'></a>

## [Data wrangling function definitions](#generalfunctions)

# Data <a id='data'></a>

<!-- ## [The COVID tracking project testing data.](#source1)
[https://covidtracking.com/api](https://covidtracking.com/api)
            -->
## [JHU CSSE case data.](#csse)
[https://systems.jhu.edu/research/public-health/ncov/](https://systems.jhu.edu/research/public-health/ncov/)
[https://github.com/CSSEGISandData/COVID-19](https://github.com/CSSEGISandData/COVID-19)

This data is split between a collection of .csv files of two different formats; first, the daily reports (global) are
separated by day, each residing in their own .csv. Additionally, the daily report files have three different formats that need to be taken into account when compiling the data. The daily report data itself contains values on the number of confirmed cases, deceased, active cases, recovered cases.

For the other format, .csv files with 'timeseries' in their filename, the data contains values for confirmed, deceased, recovered and are split between global numbers (contains United States as a whole) and numbers for the united states (statewide).
           
## [IHME hospital data](#ihme)
[http://www.healthdata.org/covid/data-downloads](http://www.healthdata.org/covid/data-downloads)

The IHME hospital data is one of the more unique datasets I've discovered with 
           
## [OWID case and test data](#owid)
[https://github.com/owid/covid-19-data](https://github.com/owid/covid-19-data)
[https://ourworldindata.org/covid-testing](https://ourworldindata.org/covid-testing)

The OWID dataset contains information regarding case and test numbers; it overlaps with the JHU CSSE 
and Testing Tracker datasets but I am going to attempt to use it in conjunction with those two because
of how there is unreliable reporting. In other words to get the bigger picture I'm looking to stitch together
multiple datasets.
           
## [OxCGRT government response data](#oxcgrt)
[https://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/master/data/OxCGRT_latest.csv](https://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/master/data/OxCGRT_latest.csv)
[https://covidtracker.bsg.ox.ac.uk/about-api](https://covidtracker.bsg.ox.ac.uk/about-api)

The OxCGRT dataset contains information regarding different government responses in regards to social
distancing measures. It measures the type of social distancing measure, whether or not they are recommended
or mandated, whether they are targeted or broad (I think geographically). 
           
## [Testing tracker data](#testtrack)
[https://www.statista.com/statistics/1109066/coronavirus-testing-in-europe-by-country/](https://www.statista.com/statistics/1109066/coronavirus-testing-in-europe-by-country/)
[https://finddx.shinyapps.io/FIND_Cov_19_Tracker/](https://finddx.shinyapps.io/FIND_Cov_19_Tracker/)

This dataset contains a time series of testing information: e.g. new (daily) tests, cumulative tests, etc. 

## [Delphi-epidata (currently not used)**](#delphi) which contains 
       Facebook surveys, google surveys, doctor visits, google health trends, quidel test data
[https://cmu-delphi.github.io/delphi-epidata/api/covidcast.html](https://cmu-delphi.github.io/delphi-epidata/api/covidcast.html)

I have not dove into this dataset too thoroughly but it contains information from facebook and google
surveys regarding COVID as well as doctor visits; the doctor visit data attempts to make distinctions between
those sick with the annual influenza and those with COVID.


# [Data regularization: making things uniform](#uniformity)

### [Intersection of countries](#country)
  
### [Time series date ranges](#time)

### [Missing Values](#missingval)

## Data wrangling function declaration <a id='generalfunctions'></a>


In [72]:
#----------------- Helper Functions for cleaning ----------------------#


def column_or_index_string_reformat(df, columns=True, index=False, dt_formats=('%m/%d/%y', '%Y-%m-%d')):
    """ Reformat column and index names. 
    
    Parameters :
    ----------
    df : Pandas DataFrame
    columns : bool
    index : bool
    
    Notes :
    -----
    Change headers of columns; this needs to be updated to account for their formatting changes. 
    This function converts strings with CamelCase, underscore and space separators to lowercase words uniformly
    separated with underscores. I.e. (hopefully!) following the correct python identifier syntax so that each column
    can be reference as an attribute if desired. 

    For more on valid Python identifiers, see:
    https://docs.python.org/3/reference/lexical_analysis.html#identifiers
    """
    if columns:
        reformatted_column_names = []
        for c in df.columns:
            # handle labels which can be cast to datetime objects
            try:
                reformatted_column_names.append(datetime.strftime(
                    datetime.strptime(c, dt_formats[0]), format=dt_formats[1]))
            except ValueError:
                reformatted_column_names.append('_'.join(re.sub('([A-Z][a-z]+)', r' \1', 
                                                         re.sub('([A-Z]+)|_|\/', r' \1', c)
                                                                .lower()).split()))
        df.columns = reformatted_column_names        
        
    if index:
        # only use only multi index dataframes where level=0 is country and level=1 is date. 
        
        
        reformatted_country_names = []
        for c in df.index.get_level_values(0):
            reformatted_country_names.append(' '.join(re.sub('([A-Z][a-z]+)', r' \1', 
                                                        re.sub('([A-Z]+)|_|\/', r' \1', c).lower())
                                                        .split()).title())
        
        reformatted_dates = pd.to_datetime(df.index.get_level_values(1)).normalize()
        restored_columns = df.index.names
        df = df.reset_index()
        df.loc[:, restored_columns[0]] = reformatted_country_names
        df.loc[:, restored_columns[1]] = reformatted_dates
        df = df.set_index(restored_columns).sort_index()
        
#     if index:
#         # only use only multi index dataframes where level=0 is country and level=1 is date. 
#         reformatted_index_names = []
#         for c in df.index.get_level_values(0):
#             # handle labels which can be cast to datetime objects
#             try:
#                 reformatted_index_names.append(datetime.strftime(
#                     datetime.strptime(c, dt_formats[0]), format=dt_formats[1]))
#             except ValueError:
#                 reformatted_index_names.append(' '.join(re.sub('([A-Z][a-z]+)', r' \1', 
#                                                         re.sub('([A-Z]+)|_|\/', r' \1', c).lower())
#                                                         .split()).title())
#         restored_column = df.index.names[0]
#         df = df.reset_index(level=0)
#         df.loc[:, restored_column] = reformatted_index_names
#         df = df.set_index([restored_column, df.index]).sort_index()
        
    return df

def csse_daily_reports_reformat():
    """ Import and concatenate all JHU CSSE daily report data from local machine. 
    """
    csv_different_formats_list = []
    
    # the actual format difference is being covered up by pd.concat which fills with Nans
    for x in glob.glob('CSSEGIS_git_case_data/csse_covid_19_data/csse_covid_19_daily_reports/*'):
        if os.path.isdir(x):
            df_list = []
            for days in glob.glob(x+'/*'):
                df = pd.read_csv(days)
                df_list.append(df)
            csv_different_formats_list.append(column_or_index_string_reformat(pd.concat(df_list, axis=0).reset_index(drop=True)))
    
    # concatenate the data
    daily_reports_df = pd.concat(csv_different_formats_list).reset_index(drop=True)
    # convert the date-like variable to datetime
    daily_reports_df.loc[:, 'last_update'] = pd.to_datetime(daily_reports_df.last_update).dt.normalize()
    # In the reporting there are duplicate values. Also, I'm aggregating by country because the other datasets
    # are not nearly as detailed. Probably should flag this somehow. 
    daily_reports_df = daily_reports_df.drop_duplicates().groupby(['country_region','last_update']).sum()
    # Reformat the location names and datetime index. Look at documentation above for details. 
    daily_reports_df = column_or_index_string_reformat(daily_reports_df, index=True, columns=True)
    # name the indices and columns for later concatenation
    daily_reports_df.index.names = ['location','date']
    daily_reports_df.columns.names = ['csse_global_daily_reports']
    return daily_reports_df
    
def csse_timeseries_reformat():
    """ Import and concatenate all JHU CSSE time series data from local machine. 
    """
    global_df_list = []

    for x in glob.glob('CSSEGIS_git_case_data/csse_covid_19_data/csse_covid_19_time_series/*_global.csv'):
        global_tmp = column_or_index_string_reformat(pd.read_csv(x))
        # only include the actual time series info; this removes latitude and 
        # longitude as well as other useless data.
        global_specific_indice_list = [1] + list(range(4, global_tmp.shape[1]))
        global_tmp = global_tmp.iloc[:,global_specific_indice_list].groupby(by='country_region').sum()
        # keep the name of the data; i.e. 'confirmed', 'deaths', etc.
        time_series_name = '_'.join(x.split('.')[0].split('_')[-2:][::-1])
        global_df_list.append(global_tmp.stack().to_frame(name=time_series_name))    
    
    # concatenate the data and name it to abide by my convention. 
    global_time_series_df = pd.concat(global_df_list, axis=1)#.reset_index(drop=True)
    global_time_series_df.index.names = ['location','date']
    global_time_series_df.columns.names = ['csse_global_timeseries']
    global_time_series_df = column_or_index_string_reformat(global_time_series_df, index=True, columns=False)

    # Repeat the steps above but for United States statewide data. 
    usa_df_list = []
    for y in glob.glob('CSSEGIS_git_case_data/csse_covid_19_data/csse_covid_19_time_series/*_US.csv'):
        usa_tmp = column_or_index_string_reformat(pd.read_csv(y))
        try:
            usa_tmp = usa_tmp.drop(columns='population')
        except: 
            pass
        usa_specific_indice_list = [6] + list(range(10, usa_tmp.shape[1]))
        usa_tmp = usa_tmp.iloc[:,usa_specific_indice_list].groupby(
            by='province_state').sum()
        time_series_name = '_'.join(y.split('.')[0].split('_')[-2:][::-1])
        usa_tmp.index.name = 'state'
        usa_df_list.append(usa_tmp.stack().to_frame(name=time_series_name))    
    
    usa_time_series_df = pd.concat(usa_df_list,axis=1)#.reset_index(drop=True)
    usa_time_series_df.index.names = ['location','date']
    usa_time_series_df.columns.names = ['csse_us_timeseries']
    usa_time_series_df = column_or_index_string_reformat(usa_time_series_df, index=True, columns=False)
    
    return global_time_series_df, usa_time_series_df


def regularize_country_names(df):
    """ Reformat column and index names. only works with with pandas MultiIndex for level=0.
    
    Parameters :
    ----------
    df : Pandas DataFrame

    Notes :
    -----
    Different datasets have different naming conventions (for countries that go by multiple names and abbreviations).
    This function imposes a convention on a selection of these country names.  
    """
    # these lists are one-to-one. countries compared via manual inspection, unfortunately. 
    mismatch_labels_bad = ['Lao People\'s Democratic Republic', 'Mainland China',
                           'Occupied Palestinian Territory','Republic of Korea', 'Korea, South', 
                           'Gambia, The ', 'UK', 
                           'USA', 'Iran (Islamic Republic of)',
                           'Bahamas, The', 'Russian Federation', 'Czech Republic', 'Republic Of Ireland',
                          'Hong Kong Sar', 'Macao Sar', 'Uk','Us',
                           'Congo ( Kinshasa)','Congo ( Brazzaville)',
                           'Cote D\' Ivoire', 'Viet Nam','Guinea- Bissau','Guinea','Usa']

    mismatch_labels_good = ['Laos','China',
                            'Palestine', 'South Korea', 'South Korea', 
                            'The Gambia', 'United Kingdom', 
                            'United States','Iran',
                            'The Bahamas','Russia','Czechia','Ireland',
                            'Hong Kong','Macao','United Kingdom', 'United States',
                            'Democratic Republic Of The Congo','Republic Of The Congo',
                            'Ivory Coast','Vietnam', 'Guinea Bissau','Guinea Bissau','United States']
    
    df = df.reset_index(level=0)
    df.loc[:,'location'] = df.loc[:,'location'].replace(to_replace=mismatch_labels_bad, value=mismatch_labels_good)
    df = df.set_index(['location', df.index])
    return df

#----------------- Helper Functions for regularization ----------------------#
def intersect_country_index(df, country_intersection):
    df_tmp = df.copy().reset_index(level=0)
    df_tmp = df_tmp[df_tmp.location.isin(country_intersection)]
    df_tmp = df_tmp.set_index(['location', df_tmp.index])
    return df_tmp 

def resample_dates(df, dates):
    df = df.loc[~df.index.duplicated(keep='first')]
    return df.reindex(pd.MultiIndex.from_product([df.index.levels[0], dates], names=['location', 'date']), fill_value=np.nan)

def make_multilevel_columns(df):
    df.columns = pd.MultiIndex.from_product([[df.columns.name], df.columns], names=['dataset', 'features'])
    return df

#----------------- Manipulation flagging ----------------------#

def flag_nan_differences(df, df_altered, suffix):
    # Use bitwise XOR to flag the values which have been changed from NaN to something else.
    # values which get mapped true -> false are those that are changed. 
    flag_df = df.isna() ^ df_altered.isna()
    z1 = tuple(flag_df.columns.get_level_values(0).tolist())
    z2 = tuple((flag_df.columns.get_level_values(1) + suffix).tolist())
    flag_df.columns = pd.MultiIndex.from_tuples(list(zip(z1,z2)),names=['dataset', 'features'])
    return flag_df


#----------------- Currently Unused ----------------------#

def pull_delphi_data(data_source=['fb-survey', 'google-survey', 'ght', 'quidel', 'quidelneg', 'doctor-visits'], 
                     daterange=pd.date_range(start="20200101",
                                             end=''.join(str(datetime.now().date()).split('-'))).strftime('%Y%m%d'),
                     **kwargs):
    """ Pull data from https://cmu-delphi.github.io/delphi-epidata/api/
        https://cmu-delphi.github.io/delphi-epidata/api/covidcast.html
    
    
    
    """
    
    for data in data_source:
        signal_dict = {'fb-survey':'smoothed_cli',
                       'google-survey':'smoothed_cli',
                       'ght':'smoothed_search',
                       'quidel':'smoothed_tests_per_device',
                       'quidelneg':'smoothed_pct_negative',
                       'doctor-visits':'smoothed_cli'}
        
        signal = signal_dict[data]
        if data=='quidelneg':
            #change the proxy for the quidel signal
            data = 'quidel'
        for days in daterange:
            resp = requests.get('https://delphi.cmu.edu/epidata/api.php?source=covidcast&data_source=doctor-visits&signal=smoothed_cli&time_type=day&geo_type=county&geo_value=*&time_values='+days)
            day_data = resp.json().get('epidata', None)
            if day_data is None:
                pass
            else:
                var_number += pd.json_normalize(day_data).size
                print(pd.json_normalize(day_data).shape)    
                
                
# date_range_2020 = pd.date_range(start="20200101",end=''.join(str(datetime.now().date()).split('-'))).strftime('%Y%m%d')
# var_number = 0 
# for days in date_range_2020:
# #     days='20200302'
#     resp = requests.get('https://delphi.cmu.edu/epidata/api.php?source=covidcast&data_source=doctor-visits&signal=smoothed_cli&time_type=day&geo_type=county&geo_value=*&time_values='+days)
#     day_data = resp.json().get('epidata', None)
#     if day_data is None:
#         pass
#     else:
#         var_number += pd.json_normalize(day_data).size
#         print(pd.json_normalize(day_data).shape)

## Data Reformatting

The following sections take the corresponding data set and reformat them such that the data
is stored in a pandas DataFrame with a multiindex; level=0 -> 'location' (country or region) and
level=1 -> date. Due to the nature of the data this is done separately for country-wide and united states-wide locations.

## JHU CSSE case data
<a id='csse'></a>
[Return to table of contents](#toc)

Tasks / to-do for this data set.

### United States COVID data

In [3]:
csse_global_daily_reports_df = csse_daily_reports_reformat().loc[:, ['confirmed','active','deaths','recovered']]

In [4]:
csse_global_timeseries_df, csse_us_timeseries_df = csse_timeseries_reformat()

In [5]:
csse_global_timeseries_df.sample(5)

,csse_global_timeseries,global_confirmed,global_deaths,global_recovered
location,date,,,
Somalia,2020-01-25,0,0,0
Tunisia,2020-04-12,707,31,43
Bangladesh,2020-02-10,0,0,0
Azerbaijan,2020-04-11,1058,11,200
Qatar,2020-01-24,0,0,0


In [6]:
# currently unused
#pd.read_csv('./CSSEGIS_git_case_data/csse_covid_19_data/UID_ISO_FIPS_LookUp_Table.csv').sample(5)

## IHME hospital data
<a id='ihme'></a>
[Return to table of contents](#toc)

[JHU CSSE](#csse) 
<font color='red'>
### Has all USA states but only 32 countries which overlap with other data; stash this dataset for now. 
</font>


In [7]:
ihme_df = column_or_index_string_reformat(pd.read_csv(
    './IHME_hospital_data/2020_04_12.02/Hospitalization_all_locs.csv').rename(columns={'location_name':'location'}))
ihme_df.loc[:, 'date'] = pd.to_datetime(ihme_df.loc[:,'date']).dt.normalize()
ihme_df = ihme_df.set_index(['location', 'date']).sort_index()

In [8]:
ihme_df.sample(5)

,,v1,allbed_mean,allbed_lower,allbed_upper,icubed_mean,icubed_lower,icubed_upper,inv_ven_mean,inv_ven_lower,inv_ven_upper,...,new_icu_upper,totdea_mean,totdea_lower,totdea_upper,bedover_mean,bedover_lower,bedover_upper,icuover_mean,icuover_lower,icuover_upper
location,date,,,,,,,,,,,,,,,,,,,,,
Wisconsin,2020-05-24,143,0.00080,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,338.101,191.0,780.275,0.0,0.0,0.0,0.0,0.0,0.0
South Dakota,2020-02-02,31,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0
Bulgaria,2020-05-24,143,0.00065,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,66.354,32.0,151.025,0.0,0.0,0.0,0.0,0.0,0.0
Liguria,2020-05-24,143,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,773.412,761.0,796.000,0.0,0.0,0.0,0.0,0.0,0.0
Oregon,2020-01-26,24,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0


## OWID case and test data
<a id='source5'></a>
[Return to table of contents](#toc)

In [9]:
owid_df = column_or_index_string_reformat(pd.read_csv('./OWID_git_and_manual_case_and_test_data/owid-covid-data.csv'))
owid_df.loc[:, 'date'] = pd.to_datetime(owid_df.loc[:, 'date']).dt.normalize()
owid_df = owid_df.set_index(['location','date']).sort_index()
owid_df = regularize_country_names(owid_df)
owid_df.columns.names = ['owid']

In [10]:
owid_df.sample(5)

,owid,iso_code,total_cases,new_cases,total_deaths,new_deaths,total_cases_per_million,new_cases_per_million,total_deaths_per_million,new_deaths_per_million,total_tests,new_tests,total_tests_per_thousand,new_tests_per_thousand,tests_units
location,date,,,,,,,,,,,,,,
Iraq,2020-04-04,IRQ,772,0,54,0,19.193,0.000,1.343,0.000,NaN,NaN,NaN,NaN,NaN
Antigua and Barbuda,2020-04-10,ATG,19,4,2,2,194.020,40.846,20.423,20.423,NaN,NaN,NaN,NaN,NaN
Benin,2020-03-18,BEN,1,0,0,0,0.082,0.000,0.000,0.000,NaN,NaN,NaN,NaN,NaN
Timor,2020-04-02,TLS,1,0,0,0,0.758,0.000,0.000,0.000,NaN,NaN,NaN,NaN,NaN
Guernsey,2020-03-26,GGY,30,7,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## OxCGRT government response data
<a id='oxcgrt'></a>
[Return to table of contents](#toc)

Pull the data using their API (for whatever reason this data set is different from the manual download).

Manual importation of data (for whatever reason this data set is different from pulling using API).

In [11]:
oxcgrt_df = column_or_index_string_reformat(pd.read_csv('./OxCGRT_response_data/OxCGRT_20200504.csv'))

In [12]:
oxcgrt_df.loc[:,'date'] = pd.to_datetime(oxcgrt_df.date,format='%Y%m%d').dt.normalize()
oxcgrt_df = oxcgrt_df.set_index(['country_name', 'date']).sort_index()
oxcgrt_df.index.names = ['location','date']
oxcgrt_df.columns.names = ['oxcgrt']

Pull the data using their API (for whatever reason this data set is different from the manual download).

In [13]:
# url_to_present_date = 'https://covidtrackerapi.bsg.ox.ac.uk/api/v2/stringency/date-range/2020-01-02/' \
#                         + str(datetime.now().date())
# response = requests.get(url_to_present_date)
# response_json = response.json()
# response_json_nested_dict = response_json['data']

# response_api_df = pd.DataFrame.from_dict({(i,j): response_json_nested_dict[i][j] 
#                            for i in response_json_nested_dict.keys() 
#                            for j in response_json_nested_dict[i].keys()},
#                        orient='index')

Because each computation requires its own unique slice, with a multiindex no less, I find it easiest
to create a DataFrame whose values are pandas multislice elements using pandas IndexSlice objects. 
These values cannot be passed at once to the testing multiindex array, but it was designed to take advantage of the .apply method. 

## Testing tracker data
<a id='testtrack'></a>
[Return to table of contents](#toc)

In [14]:
testtrack_df = pd.read_csv('./TestTracker_data/Tests_20200504.csv')
testtrack_df.loc[:, 'date'] = pd.to_datetime(testtrack_df.loc[:, 'date']).dt.normalize()
# testtrack_df.loc[:, 'date'] = pd.to_datetime(testtrack_df.loc[:, 'date'], format='%Y-%m-%d', errors='coerce')
testtrack_df = testtrack_df.set_index(['country','date']).sort_index()
testtrack_df.index.names = ['location','date']
testtrack_df.columns.names = ['test_track']
unused_columns = ['ind', 'jhu_ID.x', 'source', 'X.x', 'X.y', 'alpha2', 'alpha3',
                  'numeric', 'latitude', 'longitude', 'jhu_ID.y', 'notes']

testtrack_df = testtrack_df.drop(columns=unused_columns)
testtrack_df.sample(5)

,test_track,new_tests,tests_cumulative,penalty,population,per100k,testsPer100k
location,date,,,,,,
Greece,2020-04-20,1054,54344,1.0,10423000,521.4,521.4
Burkina Faso,2020-04-15,0,99,1.3,20903000,0.5,0.5
Philippines,2020-04-18,11757,59928,1.3,109581000,54.7,54.7
Kenya,2020-04-21,545,13784,1.3,53771000,25.6,25.6
Sri Lanka,2020-05-03,0,2082,1.3,21413000,9.7,9.7


## Delphi-epidata
<a id='delphi'></a>
[Return to table of contents](#toc)

data_source	name of upstream data source 
(e.g., fb-survey, google-survey, ght, quidel, doctor-visits)	string

signal	name of signal derived from upstream data (see notes below)	string

time_type	temporal resolution of the signal (e.g., day, week)	string

geo_type	spatial resolution of the signal (e.g., county, hrr, msa, dma, state)	string

time_values	time unit (e.g., date) over which underlying events happened	list of time values (e.g., 20200401)

geo_value	unique code for each location, depending on geo_type (county -> FIPS 6-4 code, HRR -> HRR number, MSA -> CBSA code,
DMA -> DMA code, state -> two-letter state code), or * for all	string

As of this writing, data sources have the following signals:

fb-survey signal values include raw_cli, raw_ili, raw_wcli, raw_wili, and also four additional named with raw_* replaced by smoothed_* (e.g. smoothed_cli, etc).
google-survey signal values include raw_cli and smoothed_cli.
ght signal values include raw_search and smoothed_search.
quidel signal values include smoothed_pct_negative and smoothed_tests_per_device.
doctor-visits signal values include smoothed_cli.

Delphi API data :
doctor visits : 20200201-20200429 (as of 20200503)


## Data regularization: making things uniform <a id='uniformity'></a>

## Intersection of countries in all DataFrames
<a id='country'></a>
[Return to table of contents](#toc)

The data that will be used exists in the DataFrames : 

    csse_global_daily_reports_df
    csse_global_timeseries_df
    csse_us_timeseries_df
    ihme_df
    owid_df
    oxcgrt_df
    testtrack_df
    
The index (locations) were not reformatted by default; do that now.

In [15]:
testtrack_df

test_track              new_tests  tests_cumulative  penalty  population  \
location    date                                                           
Afghanistan 2020-03-03         59                59      1.3    38928000   
            2020-03-04          0                59      1.3    38928000   
            2020-03-05         22                81      1.3    38928000   
            2020-03-06          0                81      1.3    38928000   
            2020-03-07          3                84      1.3    38928000   
...                           ...               ...      ...         ...   
Zimbabwe    2020-04-29       1247              7642      1.3    14863000   
            2020-04-30        672              8314      1.3    14863000   
            2020-05-01        977              9291      1.3    14863000   
            2020-05-02          0              9291      1.3    14863000   
            2020-05-03          0              9291      1.3    14863000   

test_track              per100k  testsPer100k  
location    date                               
Afghanistan 2020-03-03      0.2           0.2  
            2020-03-04      0.2           0.2  
            2020-03-05      0.2           0.2  
            2020-03-06      0.2           0.2  
            2020-03-07      0.2           0.2  
...                         ...           ...  
Zimbabwe    2020-04-29     51.4          51.4  
            2020-04-30     55.9          55.9  
            2020-05-01     62.5          62.5  
            2020-05-02     62.5          62.5  
            2020-05-03     62.5          62.5  

[6736 rows x 6 columns]

In [16]:
all_data = [csse_global_daily_reports_df,
    csse_global_timeseries_df,
    csse_us_timeseries_df,
    ihme_df,
    owid_df,
    oxcgrt_df,
    testtrack_df]

In [17]:
global_data = all_data[:2] + all_data[4:]

In [18]:
for i, df in enumerate(all_data):
    all_data[i] = regularize_country_names(column_or_index_string_reformat(df, index=True, columns=False))

In [19]:
for i, df in enumerate(global_data):
    global_data[i] = regularize_country_names(column_or_index_string_reformat(df, index=True, columns=False))

In [20]:
country_intersection = global_data[0].index.levels[0]
dates_union =  global_data[0].index.levels[1].unique()
for i in range(len(global_data)-1):
    country_intersection = country_intersection.intersection(global_data[i+1].index.levels[0])
    dates_union = dates_union.union(global_data[i+1].index.levels[1].unique())
    print(dates_union.min())

2020-01-22 00:00:00
2019-12-31 00:00:00
2019-12-31 00:00:00
2019-12-31 00:00:00


In [21]:
global_data_intersected = [intersect_country_index(df, country_intersection) for df in global_data]

It makes sense, because of the intersections between data; to us the u.s. time series and ihme data together but not with
the global data. The hospital data is very useful and so it may be important to look specifically at the small number of countries it contains. Regardless; by using only the global data we can keep 110 countries. 

The pruned response data has 122 countries and the testing data has 206, the intersection : 94.
These 94 countries still account for 6.6 billion people, notable missing entries are: Ethiopia, Iran. Congo, United Kingdom,
dropped because they had missing values in governement response data; i.e. they did not take all considered actions. 

## Regularization of time series dates
<a id='time'></a>
[Return to table of contents](#toc)

Want to have all time dependent data defined on the same time ranges for convenience;
this involves two steps. 1. Initialize the new dates, 2. deal with the missing values. 
Because there are already a good amount of missing values, this second step is saved until later.


With identical countries and dates (identical MultiIndex) the list of DataFrames can easier be concatenated 

## Missing Values
<a id='missingval'></a>
[Return to table of contents](#toc)

Redefining the time series range for most countries introduces a large number of missing values. To account for this, I will be replacing missing values in categorical variables with the value 'Missing' and I will be backfilling with a linear interpolant for numerical variables, so that the time series (and actions to be performed later) will be well defined. 

Next, I need to create a strategy for how to account for missing test values (i.e. when there are cases but no tests, presumably this is either a lack of reporting or the "confirmed" cases are via diagnosis rather than explicit testing).

There are two "types" of missing test values: those from errors in reporting and those from confirmation of cases via diagnosis. I distinguish between these two types via the following: errors in reporting occur when there are gaps in the testing data after the date of the first known test. The second type occurs when there are known or confirmed cases but no testing data yet exists. To account for the first type I will use forward filling, as to not overestimate 


The second part of the weighting process is by the testing numbers, as the number of cases will go up if you
test more.

Import the new testing data (much better), but still use the old dataset because it has cases normalized by population.

In [65]:
#This redefines the time series for all variables as from December 31st 2019 to the day with most recent data
normalized_global_data = [resample_dates(df, dates_union) for df in global_data_intersected]
# To keep track of which data came from where, make the columns multi level with the first level labelling the dataset.
data = pd.concat([make_multilevel_columns(df) for df in normalized_global_data], axis=1)

#OxCGRT's "flag" columns (which indicate a target or general response) are numerical but I will cast them as categorical
#so that they are not affected by the upcoming numerical feature manipulations. 
flag_columns =  data.columns.levels[1][data.columns.levels[1].str.contains('flag')]
multiindex_for_flag_columns = pd.MultiIndex.from_product([['oxcgrt'], flag_columns], names=['dataset', 'features'])
data.loc[:, multiindex_for_flag_columns] = data.loc[:, multiindex_for_flag_columns].fillna(value=-1.).astype('category')

data_numerical = data.copy().select_dtypes(include='number')

In [73]:
interpolated = data_numerical.groupby(level=0).apply(lambda x : x.interpolate(limit_direction='backward'))
interpolate_flagged = flag_nan_differences(data_numerical, interpolated, '_interpolated')

forwardfill = interpolated.groupby(level=0).fillna(method='ffill')
forwardfill_flagged = flag_nan_differences(interpolated, forwardfill, 'ffill')



#.fillna(value=-1)

In [38]:
# backfill with interpolation, forward fill the remainder; NaNs may remain if there are only missing values
# in their group. Therefore, still need to replace the remainder with something. Because so many of the features
# utilize 0, I'm going to fill the remainder of missing values with -1 because nowhere do negative values appear. 
data.loc[data_numerical.index, data_numerical.columns] = data_numerical.groupby(level=0).apply(
    lambda x : x.interpolate(limit_direction='backward')).groupby(level=0).fillna(method='ffill').fillna(value=-1)

# still_missing_values = data.loc[:, pd.IndexSlice['test_track',:]].isna().sum()#.loc[pd.IndexSlice[:, #.index.levels[1]
throw_out_these = still_missing_values.index[still_missing_values > 0]
# data = data.drop(columns=)
# These features do not seem worthwhile
data = data.drop(columns=[('owid','iso_code'),
                         ('oxcgrt','m1_wildcard'), ('oxcgrt','country_code')]
                          + throw_out_these.tolist())
# only remaining missing values are not numerical
data.loc[:, ('owid', 'tests_units')] = data.loc[:, ('owid', 'tests_units')].fillna('Missing')

In [39]:
data.sample(10)

dataset                 csse_global_daily_reports                             \
features                                confirmed   active  deaths recovered   
location     date                                                              
Ireland      2020-01-08                      1.00      0.0     0.0       0.0   
Cuba         2020-03-14                      4.25      0.0     0.0       0.0   
Saudi Arabia 2020-03-05                      5.00      0.0     0.0       0.0   
Cameroon     2020-04-28                   1705.00    842.0    58.0     805.0   
France       2020-03-13                   4970.00      0.0   103.0      16.0   
Sweden       2020-04-24                  16755.00  14184.0  2021.0     550.0   
Zimbabwe     2020-03-10                      1.00      0.0     0.0       0.0   
France       2020-02-29                    100.00      0.0     2.0      12.0   
Kazakhstan   2020-04-24                   2289.00   1709.0    20.0     560.0   
Mexico       2020-04-28                  15529.00   5009.0  1434.0    9086.0   

dataset                 csse_global_timeseries                                 \
features                      global_confirmed global_deaths global_recovered   
location     date                                                               
Ireland      2020-01-08                    0.0           0.0              0.0   
Cuba         2020-03-14                    4.0           0.0              0.0   
Saudi Arabia 2020-03-05                    5.0           0.0              0.0   
Cameroon     2020-04-28                 1705.0          58.0            915.0   
France       2020-03-13                 3681.0          79.0             12.0   
Sweden       2020-04-24                17567.0        2152.0           1005.0   
Zimbabwe     2020-03-10                    0.0           0.0              0.0   
France       2020-02-29                  100.0           2.0             12.0   
Kazakhstan   2020-04-24                 2482.0          25.0            604.0   
Mexico       2020-04-28                16752.0        1569.0          11423.0   

dataset                          owid                         ...  \
features                  total_cases new_cases total_deaths  ...   
location     date                                             ...   
Ireland      2020-01-08      0.000000       0.0          0.0  ...   
Cuba         2020-03-14      4.000000       1.0          0.0  ...   
Saudi Arabia 2020-03-05      3.666667       3.0          0.0  ...   
Cameroon     2020-04-28   1016.000000       0.0         42.0  ...   
France       2020-03-13   2876.000000     595.0         61.0  ...   
Sweden       2020-04-24  14385.000000     563.0       1540.0  ...   
Zimbabwe     2020-03-10      1.000000       1.0          0.0  ...   
France       2020-02-29     57.000000      19.0          2.0  ...   
Kazakhstan   2020-04-24   1735.000000      81.0         19.0  ...   
Mexico       2020-04-28   8261.000000     764.0        686.0  ...   

dataset                           oxcgrt                               \
features                stringency_index stringency_index_for_display   
location     date                                                       
Ireland      2020-01-08             0.00                         0.00   
Cuba         2020-03-14            11.11                        11.11   
Saudi Arabia 2020-03-05            11.11                        11.11   
Cameroon     2020-04-28            63.88                        63.88   
France       2020-03-13            42.86                        42.86   
Sweden       2020-04-24            47.35                        47.35   
Zimbabwe     2020-03-10            13.89                        13.89   
France       2020-02-29            21.82                        21.82   
Kazakhstan   2020-04-24            74.88                        74.88   
Mexico       2020-04-28            68.38                        68.38   

dataset                                          \
featur

In [40]:
data.to_csv('data.csv')

## Repeat of the above calculations for United States only data.

<font color='red'>
unfinished as of now
</font>

The United States' data merits separate investigation 1. because of the case number 2. because the IHME dataset is only really
properly defined for the statewide description of the U.S. 

In [27]:
csse_us_timeseries_df

csse_us_timeseries   US_confirmed  US_deaths
location date                               
Alabama  2020-01-22             0          0
         2020-01-23             0          0
         2020-01-24             0          0
         2020-01-25             0          0
         2020-01-26             0          0
...                           ...        ...
Wyoming  2020-04-29           545          7
         2020-04-30           559          7
         2020-05-01           566          7
         2020-05-02           579          7
         2020-05-03           586          7

[5974 rows x 2 columns]

In [28]:
ihme_df

v1  allbed_mean  allbed_lower  allbed_upper  \
location date                                                       
Abruzzo  2020-01-03    1          0.0           0.0           0.0   
         2020-01-04    2          0.0           0.0           0.0   
         2020-01-05    3          0.0           0.0           0.0   
         2020-01-06    4          0.0           0.0           0.0   
         2020-01-07    5          0.0           0.0           0.0   
...                  ...          ...           ...           ...   
Wyoming  2020-07-31  211          0.0           0.0           0.0   
         2020-08-01  212          0.0           0.0           0.0   
         2020-08-02  213          0.0           0.0           0.0   
         2020-08-03  214          0.0           0.0           0.0   
         2020-08-04  215          0.0           0.0           0.0   

                     icubed_mean  icubed_lower  icubed_upper  inv_ven_mean  \
location date                                                                
Abruzzo  2020-01-03          0.0           0.0           0.0           0.0   
         2020-01-04          0.0           0.0           0.0           0.0   
         2020-01-05          0.0           0.0           0.0           0.0   
         2020-01-06          0.0           0.0           0.0           0.0   
         2020-01-07          0.0           0.0           0.0           0.0   
...                          ...           ...           ...           ...   
Wyoming  2020-07-31          0.0           0.0           0.0           0.0   
         2020-08-01          0.0           0.0           0.0           0.0   
         2020-08-02          0.0           0.0           0.0           0.0   
         2020-08-03          0.0           0.0           0.0           0.0   
         2020-08-04          0.0           0.0           0.0           0.0   

                     inv_ven_lower  inv_ven_upper  ...  new_icu_upper  \
location date                                      ...                  
Abruzzo  2020-01-03            0.0            0.0  ...            0.0   
         2020-01-04            0.0            0.0  ...            0.0   
         2020-01-05            0.0            0.0  ...            0.0   
         2020-01-06            0.0            0.0  ...            0.0   
         2020-01-07            0.0            0.0  ...            0.0   
...                            ...            ...  ...            ...   
Wyoming  2020-07-31            0.0            0.0  ...            0.0   
         2020-08-01            0.0            0.0  ...            0.0   
         2020-08-02            0.0            0.0  ...            0.0   
         2020-08-03            0.0            0.0  ...            0.0   
         2020-08-04            0.0            0.0  ...            0.0   

                     totdea_mean  totdea_lower  totdea_upper  bedover_mean  \
location date                                                                
Abruzzo  2020-01-03         0.00           0.0         0.000           0.0   
         2020-01-04         0.00           0.0         0.000           0.0   
         2020-01-05         0.00           0.0         0.000           0.0   
         2020-01-06         0.00           0.0         0.000           0.0   
         2020-01-07         0.00           0.0         0.000           0.0   
...                          ...           ...           ...           ...   
Wyoming  2020-07-31        34.22           0.0       231.025           0.0   
         2020-08-01        34.22           0.0       231.025           0.0   
         2020-08-02        34.22           0.0       231.025           0.0   
         2020-08-03        34.22           0.0       231.025           0.0   
         2020-08-04        34.22           0.0       231.025           0.0   

                     bedover_lower  bedover_upper  icuover_mean  \
location date                                                     
Abruzzo  2

In [29]:
usa_data = [
    csse_us_timeseries_df,
    ihme_df,
    owid_df,
    oxcgrt_df,
    testtrack_df]